In [39]:
import torch

# Set device to MPS
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")




In [40]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import ollama
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import psutil

In [41]:
from langchain.docstore.document import Document
file_path = "document/raw_data.txt"  

with open(file_path, 'r', encoding='utf-8') as file:
    doc_text = file.read()

doc = Document(page_content=doc_text)

In [42]:
doc

Document(metadata={}, page_content='highway traffic act\n     \ninterpretation, general  \ndefinitions  \n1 (1) in this act,  \n“ambulance” includes,  \n  (a) an ambulance as defined in the ambulance act,  \n  (b) a cardiac arrest emergency vehicle operated by or under the authority of a hospital, and  \n  (c) an emergency response vehicle, other than an ambulance as defined in the ambulance act, operated by an ambulance service that is used to provide emergency response services, and that has been assigned an emergency response vehicle number by the ministry of health and long-term care; \n“bicycle” includes a tricycle, a unicycle and a power-assisted bicycle but does not include a motor assisted bicycle; \n“built-up area” means a territory contiguous to a highway not within a local municipality, other than a local municipality that had the status of a township on december 31, 2002 and, but for the enactment of the municipal act, 2001, would have had the status of a township on januar

In [43]:
text_spliter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30, separator = "\n")
docs = text_spliter.split_documents(documents = [doc])

In [44]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"   # It maps sentences & paragraphs to a 384 dimensional dense vector space
model_kwargs = {'device': device} 
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs=model_kwargs) #model_kwargs=model_kwargs

In [45]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [46]:
vectorstore.save_local("faiss_index")

In [ ]:
embedding_vectorestore = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization=True)

In [48]:
retriever = embedding_vectorestore.as_retriever(search_type="similarity")

In [49]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")

In [50]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [66]:
response = qa.invoke("At what age is a person allowed to drive an off-road vehicle under the Act?")
response

{'query': 'At what age is a person allowed to drive an off-road vehicle under the Act?',
 'result': 'According to section 4(1) of the Act, no owner of an off-road vehicle shall permit a child under the age of twelve to drive the vehicle. However, there are exceptions where the child can drive under close supervision of an adult on land occupied by the vehicle owner (section 4(2)(a)).'}

In [65]:
response = qa.invoke("Is Trail permit required?")
response

{'query': 'Is Trail permit required?',
 'result': 'Yes, according to section 2.1 (1), no person shall drive a motorized snow vehicle upon a prescribed trail except under the authority of, and in accordance with, a trail permit for the motorized snow vehicle issued under subsection (2).'}

In [54]:
response = qa.invoke("Does the municiple maintains any record of population?")
response

{'query': 'Does the municiple maintains any record of population?',
 'result': "Based on the provided context, I couldn't find any information that suggests the municipality is required to maintain a record of population under this act or regulation. The relevant sections (e.g., 225(1), 205(e)) appear to pertain to record keeping and reporting requirements for the Ministry, rather than municipal population records.\n\nHowever, it's worth noting that some municipalities may choose to collect and maintain demographic data, including population information, for internal planning or research purposes. But this is not explicitly required under the provided regulations."}

In [55]:
response = qa.invoke("What are the penalties for careless driving?")
response

{'query': 'What are the penalties for careless driving?',
 'result': "According to sections (3) and (5) of the provided context, the penalties for careless driving causing bodily harm or death are:\n\n* A fine of not less than $400 and not more than $2,000\n* Imprisonment for a term of not more than six months\n* Suspension of driver's licence or permit for a period of not more than two years\n\nAdditionally, according to section (4) of the same context, the penalties for careless driving are:\n\n* A fine of not less than $100 and not more than $200\n* Imprisonment for a term of not more than 30 days"}

In [56]:
response = qa.invoke("What are the penalties for careless driving?")
response

{'query': 'What are the penalties for careless driving?',
 'result': "The penalty for careless driving causing bodily harm or death is a fine of not less than $400 and not more than $2,000 or imprisonment for a term of not more than six months, or to both.\n\nAdditionally, the driver's licence or permit may be suspended for a period of not more than two years.\n\nIt's also worth noting that there are different penalties for commercial motor vehicle drivers who fail to stop when required under subsection (1) to do so, which includes a fine of not less than $200 and not more than $20,000, and the suspension of their driver’s licence for a period of not more than 30 days."}

In [57]:
import pandas as pd

df = pd.read_excel('TrainingDataset.xlsx')


In [58]:
from sentence_transformers import SentenceTransformer, util
sentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2')


In [59]:
# Function to compute ROUGE-L score between correct answer and model's answer
def compute_rouge_l(correct_answer, model_answer):
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    score = scorer.score(correct_answer, model_answer)
    return score["rougeL"].fmeasure  # Return ROUGE-L F1 score


In [60]:
import psutil
import time
from rouge_score import rouge_scorer
from memory_profiler import memory_usage

similarities = []
responses = []
similarities = []
answer_cosine_similarities = []
rouge_l_scores = []
times = []
memory_usage_MB = []

# Get the current process for memory tracking
process = psutil.Process()


In [61]:
def getAnswer(question):
    return qa.invoke(question)

In [62]:
for index, row in df.iterrows():
    question, true_answer = row['Question'], row['Answer']

    start_time = time.time()
    initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
    mem_usage_before = memory_usage(process.pid, interval=0.1, timeout=1)

    generated_response = getAnswer(question)
    end_time = time.time()
    mem_usage_after = memory_usage(process.pid, interval=0.1, timeout=1)
    memory_used = max(mem_usage_after) - min(mem_usage_before)
    
    generated_response = generated_response.get('result', "No response generated")  # Default message if 'result' is not in dictionary
 
    # Ensure the response is in the correct format for encoding
    if isinstance(generated_response, str):
        responses_to_encode = [generated_response]
    elif isinstance(generated_response, list):
        responses_to_encode = generated_response
    else:
        print("Unexpected response type:", type(generated_response))
        continue

    rouge_l_score = compute_rouge_l(true_answer, generated_response)  
    # Encode both the generated response and the true answer
    response_embedding = sentenceTransformer.encode(responses_to_encode)
    answer_embedding = sentenceTransformer.encode([true_answer])

    # Compute cosine similarity
    similarity = util.cos_sim(response_embedding, answer_embedding)
    similarities.append(similarity.item())  # Store the scalar similarity value

    # Compute time taken by the model to give each answer
    times.append(end_time - start_time)  

    # Compute memory usage by each answer
    memory_usage_MB.append(memory_used)  

    responses.append(generated_response)

    rouge_l_scores.append(rouge_l_score)

KeyboardInterrupt: 

In [ ]:
# Add the results as new columns in the DataFrame
df['Model_Response'] = responses
df['Cosine_Similarity'] = similarities
df['ROUGE_L_Score'] = rouge_l_scores
df['Time_Taken_Sec'] = times
df['Memory_Usage_MB'] = memory_usage_MB
# Optionally, add the similarities back to the DataFrame
df.to_excel('Metrics_llama3.2.xlsx', index=False)